# __10 Statistical Natural Language Processing for Sentiment Analysis__

Generally, sentiment analysis is performed based on the processing of natural
language, the analysis of text and computational linguistics. Although data can come
from different data sources, in this chapter we will analyze sentiment in text data,
using two particular text data examples: one from film critics, where the text is highly
structured and maintains text semantics; and another example coming from social
networks (tweets in this case), where the text can show a lack of structure and users
may use (and abuse!) text abbreviations.

In the following sections, we will review some basic mechanisms required to
perform sentiment analysis. In particular, we will analyze the steps required for
data cleaning (that is, removing irrelevant text items not associated with sentiment
information), producing a general representation of the text, and performing some
statistical inference on the text represented to determine positive and negative senti-
ments.

Although the scope of sentiment analysis may introduce many aspects to be ana-
lyzed, in this chapter and for simplicity, we will analyze binary sentiment analysis
categorization problems. We will thus basically learn to classify positive against
negative opinions from text data. The scope of sentiment analysis is broader, and it
includes many aspects that make analysis of sentiments a challenging task. Some
interesting open issues in this topic are as follows:

+ Identification of sarcasm: sometimes without knowing the personality of the per-
    son, you do not know whether “bad” means bad or good.
+ Lack of text structure: in the case of Twitter, for example, it may contain abbre-
    viations, and there may be a lack of capitals, poor spelling, poor punctuation, and
    poor grammar, all of which make it difficult to analyze the text.
+ Many possible sentiment categories and degrees: positive and negative is a simple
    analysis, one would like to identify the amount of hate there is inside the opinion,
    how much happiness, how much sadness, etc.
+ Identification of the object of analysis: many concepts can appear in text, and how
    to detect the object that the opinion is positive for and the object that the opinion is
    negative for is an open issue. For example, if you say “She won him!”, this means
    a positive sentiment for her and a negative sentiment for him, at the same time.
+ Subjective text: another open challenge is how to analyze very subjective sentences
    or paragraphs. Sometimes, even for humans it is very hard to agree on the sentiment
    of these highly subjective texts.

## __Data Cleaning__

The main task of data cleaning is to remove
those characters considered as noise in the data mining process. For instance, comma
or colon characters. Of course, in each particular data mining problem different char-
acters can be considered as noise, depending on the final objective of the analysis. In
our case, we are going to consider that all punctuation characters should be removed,
including other non-conventional symbols.

In [2]:
raw_docs = [
    'Here are some very simple basic sentences.',
    'They won`t be very interesting, I`m afraid.',
    'The point of these examples i to _learn how basoc text\ cleaning works_ on *very simple* data. '
]

The first step consists of defining a list with all word-vectors in the text.
`NLTK` makes it easy to convert documents-as-trings into word-vectors, a process
called tokenizing.

In [4]:
from nltk.tokenize import word_tokenize

tokenized_docs = [word_tokenize(doc) for doc in raw_docs]

In [5]:
print(tokenized_docs)

[['Here', 'are', 'some', 'very', 'simple', 'basic', 'sentences', '.'], ['They', 'won', '`', 't', 'be', 'very', 'interesting', ',', 'I', '`', 'm', 'afraid', '.'], ['The', 'point', 'of', 'these', 'examples', 'i', 'to', '_learn', 'how', 'basoc', 'text\\', 'cleaning', 'works_', 'on', '*', 'very', 'simple', '*', 'data', '.']]


Thus, for each line of text in raw_docs, word_tokenize function will set
the list of word-vectors. Now we can search the list for punctuation symbols, for
instance, and remove them. There are many ways to perform this step. Let us see
one possible solution using the `String` library.

In [6]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [11]:
import re
import string

regex = re.compile(f'[{re.escape(string.punctuation)}]')
tokenized_docs_no_punctuation = []

for review in tokenized_docs:
    new_review = []
    for token in review:
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            new_review.append(new_token)
    tokenized_docs_no_punctuation.append(new_review)

print(tokenized_docs_no_punctuation)

[['Here', 'are', 'some', 'very', 'simple', 'basic', 'sentences'], ['They', 'won', 't', 'be', 'very', 'interesting', 'I', 'm', 'afraid'], ['The', 'point', 'of', 'these', 'examples', 'i', 'to', 'learn', 'how', 'basoc', 'text', 'cleaning', 'works', 'on', 'very', 'simple', 'data']]


Another important step in many data mining systems for text analysis consists of
stemming and lemmatizing. Morphology is the notion that words have a root form.
If you want to get to the basic term meaning of the word, you can try applying
a stemmer or lemmatizer. This step is useful to reduce the dictionary size and the
posterior high-dimensional and sparse feature spaces. NLTK provides different ways
of performing this procedure. In the case of running the porter.stem(word)
approach, the output is shown next.

In [13]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

porter = PorterStemmer()
snowball = SnowballStemmer('english')
wordnet = WordNetLemmatizer()

preprocessed_docs = []
for doc in tokenized_docs_no_punctuation:
    final_doc = []
    for word in doc:
        final_doc.append(porter.stem(word))
    preprocessed_docs.append(final_doc)
    
print(tokenized_docs_no_punctuation)
print(preprocessed_docs)

[['Here', 'are', 'some', 'very', 'simple', 'basic', 'sentences'], ['They', 'won', 't', 'be', 'very', 'interesting', 'I', 'm', 'afraid'], ['The', 'point', 'of', 'these', 'examples', 'i', 'to', 'learn', 'how', 'basoc', 'text', 'cleaning', 'works', 'on', 'very', 'simple', 'data']]
[['here', 'are', 'some', 'veri', 'simpl', 'basic', 'sentenc'], ['they', 'won', 't', 'be', 'veri', 'interest', 'I', 'm', 'afraid'], ['the', 'point', 'of', 'these', 'exampl', 'i', 'to', 'learn', 'how', 'basoc', 'text', 'clean', 'work', 'on', 'veri', 'simpl', 'data']]


This kind of approaches are very useful in order to reduce the exponential number
of combinations of words with the same meaning and match similar texts. Words
such as “interest” and “interesting” will be converted into the same word “interest”
making the comparison of texts easier, as we will see later.


## __Text Representation__

In the previous section we have analyzed different techniques for data cleaning, stem-
ming, and lemmatizing, and filtering the text to remove other unnecessary tags for
posterior text analysis. In order to analyze sentiment from text, the next step consists
of having a representation of the text that has been cleaned.
Although different reprresentations of text exist, the most common ones are variants of Bag of Words (BoW) models.

he basic idea is to think about word frequencies. If we can define a
dictionary of possible different words, the number of different existing words will
define the length of a feature space to represent each text.



Next, we will see a particular case of bag of words, the Vector Space Model of
text: TF–IDF (term frequency–inverse distance frequency). First, we need to count
the terms per document, which is the term frequency vector. See a code example
below.

In [17]:
mydoclist = [
    'Mireia loves me more than Hector loves me',
    'Sergio likes me more than Mireia loves me',
    'HE likes basketball more than football'
]

from collections import Counter

for doc in mydoclist:
    tf = Counter()
    for word in doc.split():
        tf[word] += 1
    print(tf.items())

dict_items([('Mireia', 1), ('loves', 2), ('me', 2), ('more', 1), ('than', 1), ('Hector', 1)])
dict_items([('Sergio', 1), ('likes', 1), ('me', 2), ('more', 1), ('than', 1), ('Mireia', 1), ('loves', 1)])
dict_items([('HE', 1), ('likes', 1), ('basketball', 1), ('more', 1), ('than', 1), ('football', 1)])


Let us call this a first stab at representing documents quantitatively, just by their
word counts (also thinking that we may have previously filtered and cleaned the text
using previous approaches). Here we show an example for computing the feature
vector based on word frequencies.

In [18]:
def build_lexicon(corpus):
    '''Define a set with all possible words included in all the
    sentences or corpus.
    '''
    lexicon = set()
    for doc in corpus:
        lexicon.update([word for word in doc.split()])
    return lexicon

def freq(term, document):
    return document.split().count(term)

def tf(term, document):
    return freq(term, document)


In [22]:
vocabulary = build_lexicon(mydoclist)
doc_term_matrix = []

print('out vocabulary vector is [' + ', '.join(list(vocabulary)) + ']')

for doc in mydoclist:
    print('the doc is "' + doc + '"')
    tf_vector = [tf(word, doc) for word in vocabulary]
    tf_vector_string = ', '.join(format(freq, 'd') for freq in tf_vector)
    print(f'the tf vector for Document {mydoclist.index(doc) + 1} is [{tf_vector_string}]')
    doc_term_matrix.append(tf_vector)

print('all combined, here is our master document term matrix: ')
print(doc_term_matrix)

out vocabulary vector is [Hector, Sergio, Mireia, more, likes, football, me, HE, loves, than, basketball]
the doc is "Mireia loves me more than Hector loves me"
the tf vector for Document 1 is [1, 0, 1, 1, 0, 0, 2, 0, 2, 1, 0]
the doc is "Sergio likes me more than Mireia loves me"
the tf vector for Document 2 is [0, 1, 1, 1, 1, 0, 2, 0, 1, 1, 0]
the doc is "HE likes basketball more than football"
the tf vector for Document 3 is [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1]
all combined, here is our master document term matrix: 
[[1, 0, 1, 1, 0, 0, 2, 0, 2, 1, 0], [0, 1, 1, 1, 1, 0, 2, 0, 1, 1, 0], [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1]]


Now, every document is in the same feature space, meaning that we can represent
the entire corpus in the same dimensional space. Once we have the data in the
same feature space, we can start applying some machine learning methods: learning,
classifying, clustering, and so on. But actually, we have a few problems. Words are
not all equally informative. If words appear too frequently in a single document,
they are going to muck up our analysis. We want to perform some weighting of these
term frequency vectors into something a bit more representative. That is, we need to
do some vector normalizing. One possibility is to ensure that the L2 norm of each
vector is equal to 1.

In [25]:
import math
import numpy as np

def l2_normalizer(vec):
    denom = np.sum([el ** 2 for el in vec])
    return [(el / math.sqrt(denom)) for el in vec]

doc_term_matrix_l2 = []
for vec in doc_term_matrix:
    doc_term_matrix_l2.append(l2_normalizer(vec))
    
print('a regular old document term matrix: ')
print(np.matrix(doc_term_matrix))

print('\nA document term matrix with row-wise L2 norm: ')
print(np.matrix(doc_term_matrix_l2))

a regular old document term matrix: 
[[1 0 1 1 0 0 2 0 2 1 0]
 [0 1 1 1 1 0 2 0 1 1 0]
 [0 0 0 1 1 1 0 1 0 1 1]]

A document term matrix with row-wise L2 norm: 
[[0.28867513 0.         0.28867513 0.28867513 0.         0.
  0.57735027 0.         0.57735027 0.28867513 0.        ]
 [0.         0.31622777 0.31622777 0.31622777 0.31622777 0.
  0.63245553 0.         0.31622777 0.31622777 0.        ]
 [0.         0.         0.         0.40824829 0.40824829 0.40824829
  0.         0.40824829 0.         0.40824829 0.40824829]]


You can see that we have scaled down the vectors so that each element is between
[0, 1]. This will avoid getting a diminishing return on the informative value of a word
massively used in a particular document. For that, we need to scale down words that
appear too frequently in a document.

Finally, we have a final task to perform. Just as not all words are equally valuable
within a document, not all words are valuable across all documents. We can try
reweighting every word by its inverse document frequency.